<a href="https://colab.research.google.com/github/MohibShaikh/Action-Detector-using-LSTM/blob/main/Action_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *1. Installing Dependencies*

In [ ]:
!pip install tensorflow
!pip install opencv-python
!pip install mediapipe
!pip install scikit-learn
!pip install matplotlib

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time
import mediapipe as mp

### *2. Keypoints using MP Holistic*

In [ ]:
mp_holistic = mp.solutions.holistic       # Holistic Model
mp_drawing = mp.solutions.drawing_utils   # Drawing Utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flag.writeable = False
    results = model.process(image)
    image.flag.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [ ]:
cap = cv2.VideoCapture(2)
while cap.isOpened():
    ret,frame = cap.read()
    cv2.imshow('OpenCV Feed',frame)

    if cv2.waitkey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()